In [1]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.integrate import trapz, cumtrapz
import matplotlib.pyplot as plt

In [2]:
def nearest(vector,value):
    res=(np.abs(vector - value)).argmin()
    return res

In [56]:
# compute BHAR function from catalogue's galaxies + Aird's p(sBHAR)
#logms_sample=[8.75,9.25,9.75,10.25,10.75,11.25]
#p=[np.loadtxt("../../../GalaxyProjects/Discrete-Statistical-Semi-Empirical-Model/DREAM/Data/Aird+2019/p_lambda_z=0.3_Ms=%.2f.txt"%logMS) for logMS in logms_sample]
#cum_int=[ cumtrapz(10.**p[i][:,1], p[i][:,0], initial=0.) for i in range(len(logms_sample)) ]
#interps=[ interp1d(cum_int[i], p[i][:,0], fill_value="extrapolate") for i in range(len(logms_sample)) ]
#maxs=[np.max(cum_int[i]) for i in range(len(logms_sample)) ]
#mins=[np.min(cum_int[i]) for i in range(len(logms_sample)) ]
#idx=nearest(z,0.3)
#sBHAR=[]
#for igal in range(nhalo):
#    if 8.5<logmstar_integrated[igal,idx]<=9.:
#        index=0
#    elif 9<logmstar_integrated[igal,idx]<=9.5:
#        index=1
#    elif 9.5<logmstar_integrated[igal,idx]<=10.:
#        index=2
#    elif 10.<logmstar_integrated[igal,idx]<=10.5:
#        index=3
#    elif 10.5<logmstar_integrated[igal,idx]<=11.:
#        index=4
#    elif 11.<logmstar_integrated[igal,idx]:#<=11.5:
#        index=5
#    else:
#        sBHAR.append(-66.)
#        continue
#    randn=np.random.uniform(mins[index],maxs[index])
#    sBHAR.append(float(interps[index](randn)))
#sBHAR=np.array(sBHAR)
#BHAR=sBHAR+logmstar_integrated[:,idx]

NameError: name 'nhalo' is not defined

In [ ]:
# compute BHAR function from catalogue's galaxies + Aird's p(BHAR/SFR)
logms_sample=[8.75,9.25,9.75,10.25,10.75,11.25]
p=[np.loadtxt("../../../GalaxyProjects/Discrete-Statistical-Semi-Empirical-Model/DREAM/Data/Aird+2020/p_BHAR_SFR_z=0.3_Ms=%.2f.txt"%logMS) for logMS in logms_sample]
cum_int=[ cumtrapz(10.**p[i][:,1], p[i][:,0], initial=0.) for i in range(len(logms_sample)) ]
interps=[ interp1d(cum_int[i], p[i][:,0], fill_value="extrapolate") for i in range(len(logms_sample)) ]
maxs=[np.max(cum_int[i]) for i in range(len(logms_sample)) ]
mins=[np.min(cum_int[i]) for i in range(len(logms_sample)) ]
idx=nearest(z,0.3)
BHAR=[]
for igal in range(nhalo):
    if logmstar_integrated[igal,idx]<=9.:
        index=0
    elif 9<logmstar_integrated[igal,idx]<=9.5:
        index=1
    elif 9.5<logmstar_integrated[igal,idx]<=10.:
        index=2
    elif 10.<logmstar_integrated[igal,idx]<=10.5:
        index=3
    elif 10.5<logmstar_integrated[igal,idx]<=11.:
        index=4
    elif 11.<logmstar_integrated[igal,idx]:#<=11.5:
        index=5
    else:
        BHAR.append(-66.)
        continue
    randn=np.random.uniform(mins[index],maxs[index])
    BHAR.append(float(interps[index](randn)))
BHAR=np.array(BHAR)
BHAR=BHAR+sfrlog_cat[:,idx]

In [ ]:
#plt.hist(sBHAR[sBHAR>-66.])
plt.hist(BHAR[BHAR>-50.])
plt.yscale("log")

In [ ]:
mbhdotlog=np.arange(-6, 2, 0.1)
phi_BHAR=np.histogram(BHAR,bins=mbhdotlog)[0] / 0.1 / volume
plt.plot(mbhdotlog[:-1]+0.05, np.log10(phi_BHAR))
plt.xlim(-5.5,1.2)
plt.ylim(-5.2,0.2)

In [50]:
z=np.arange(0.,6.,0.1)
logms=np.arange(8,12,0.1)
mbhdotlog=np.arange(-10, 2, 0.1)
red_sample=[0.3, 0.75, 1.25, 1.75, 2.25, 2.75]
matrix=np.zeros((logms.size, z.size, mbhdotlog.size))
for iz,z in enumerate(z):
    Z=red_sample[nearest(red_sample,z)]
    if Z==0.3 or Z==0.75:
        logms_sample=[8.75,9.25,9.75,10.25,10.75,11.25]
    elif Z==1.25:
        logms_sample=[9.25,9.75,10.25,10.75,11.25]
    elif Z==1.75:
        logms_sample=[9.75,10.25,10.75,11.25]
    elif Z==2.25 or Z==2.75:
        logms_sample=[10.25,10.75,11.25]
    p=[np.loadtxt("../../../GalaxyProjects/Discrete-Statistical-Semi-Empirical-Model/DREAM/Data/Aird+2020/p_BHAR_SFR_z={:.2f}_Ms={:.2f}.txt".format(Z,logMS)) for logMS in logms_sample]
    #cum_int=[ cumtrapz(10.**p[i][:,1], p[i][:,0], initial=0.) for i in range(len(logms_sample)) ]
    #interps=[ interp1d(cum_int[i], p[i][:,0], fill_value="extrapolate") for i in range(len(logms_sample)) ]
    #maxs=[np.max(cum_int[i]) for i in range(len(logms_sample)) ]
    #mins=[np.min(cum_int[i]) for i in range(len(logms_sample)) ]
    p_interpolated=np.array([ interp1d(p[i][:,1], p[i][:,0], fill_value="extrapolate")(mbhdotlog) for i in range(len(logms_sample)) ])
    for imbh,Mbhdotlog in enumerate(mbhdotlog):
        matrix[:,iz,imbh]=interp1d( logms_sample, p_interpolated[:,imbh], fill_value="extrapolate" )(logms)
#np.save("Aird_p_BHAR_SFR/p_BHAR_SFR_matrix.npy",matrix)

In [52]:
cum_prob=matrix.copy()#np.zeros((logms.size, z.size, mbhdotlog.size))
for iz in range(z.size):
    for ims in range(logms.size):
        cum_prob[ims,iz,:]=cumtrapz(10.**matrix[ims,iz,:],mbhdotlog,initial=0.)
#np.save("Aird_p_BHAR_SFR/Aird_cum_prob.npy",cum_prob)

/Users/haofu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in power
  after removing the cwd from sys.path.
